In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import scipy
%matplotlib inline

In [ ]:
import glob
# import imageio
from skimage import color
from skimage import io

fruit_types = glob.glob('../datasets/fruits-360/Training/*')
fruit_types.sort
fruit_types
X_train = []
y_train = []
for i in range(len(fruit_types)):
    fruit_images = glob.glob(fruit_types[i] + '/*')
    for image in fruit_images:
        X_train += [color.rgb2gray(io.imread(image))]
        y_train.append(i)
plt.imshow(X_train[0])

In [ ]:
test_types = glob.glob('../datasets/fruits-360/Validation/*')
test_types.sort
test_types
X_test = []
y_test = []
for i in range(len(test_types)):
    test_images = glob.glob(test_types[i] + '/*')
    for image in test_images:
        X_test += [color.rgb2gray(io.imread(image))]
        y_test.append(i)
plt.imshow(X_test[0])

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of X_test: {}".format(X_test.shape))

In [ ]:
X_train = X_train.reshape(29228, 10000).astype('float32')
X_test = X_test.reshape(9837, 10000).astype('float32')

In [ ]:
X_test[0]

In [ ]:
# X_train /= 255
# X_test /= 255

In [ ]:
y_train = keras.utils.to_categorical(y_train, 60)
y_test = keras.utils.to_categorical(y_test, 60)

In [ ]:
model = Sequential()
model.add(Dense(10000, activation='tanh', input_shape=(10000,)))
model.add(Dense(2500, activation='relu'))
model.add(Dense(625, activation='relu'))
model.add(Dense(60, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_test, y_test))